In [1]:
import torch
import torchvision

from torchvision.transforms import v2
from model_bruno import *
from utils import *
from time import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
transforms = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    v2.ColorJitter(25, 25),
    v2.ToDtype(torch.float32, True),
])

STL10_set = torchvision.datasets.STL10(root='C:/Users/Bruno/OneDrive/Documents/Datasets/STL10', split='train', transform=transforms, download=True)
STL10_loader = torch.utils.data.DataLoader(STL10_set, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

  0%|          | 720896/2640397119 [00:08<8:52:22, 82637.57it/s] 


KeyboardInterrupt: 

In [3]:
num_epochs = 50
num_clusters = 10
num_classes = 10
area_penalty = False

model = AttModel(num_classes=num_classes, num_clusters=num_clusters)
model = model.to(device)
opt = torch.optim.Adam(model.parameters(), 1e-4)

c:\Users\Bruno\miniconda3\envs\pytorch\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [2]:
model.train()
for epoch in range(num_epochs):
    tic = time()
    avg_loss = 0
    avg_acc = 0
    for x,y in STL10_loader:
        x = x.to(device)
        y = y.to(device)

        pred, cluster_atts, bboxes = model(x)
        loss = torch.nn.functional.cross_entropy(pred, y)

        if area_penalty:
            loss += torch.sum((bboxes[:,-1]*bboxes[:,-2]))/num_clusters**2

        opt.zero_grad()
        loss.backward()
        opt.step()
        
        avg_loss += float(loss) / len(STL10_loader)
        avg_acc += (y == pred.argmax(1)).float().mean() / len(STL10_loader)   
    
    toc = time()
    
    print(f'Epoch {epoch+1}/{num_epochs} - {toc-tic:.0f}s - Avg loss: {avg_loss} - Avg acc: {avg_acc}')

NameError: name 'model' is not defined